In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from sklearn import tree
import copy
import calendar
#import seaborn as sns

import plotting
import dataset_fctns
import modelling_fctns
#from dwd_phenpy import Phenology_set

#import openeo
#import ee
## Trigger the authentication flow.
#ee.Authenticate()#(auth_mode='localhost')
# Initialize the library.
#ee.Initialize(project='ee-martinparker637')

In [2]:
Maize_set = Phenology_set("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\PH_Jahresmelder_Landwirtschaft_Kulturpflanze_Mais_1936_2023_hist.txt")
Maize_set.drop_columns(['Unnamed: 9'])
Maize_set.phase_order_name([10, 12, 67, 65, 5, 6, 19, 20, 21, 24, ])

In [3]:
Maize_set.get_mean_T('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\tas_hyras_5_1951_2020_v5-0_de.nc')
Maize_set.make_input_array()

project to new coords
interpolate driver to station locations


In [4]:
tp=3
resps = [lambda x: 0.053*modelling_fctns.Wang_Engel_Temp_response(x, 8, 28, 36), lambda x: 0.055*modelling_fctns.Wang_Engel_Temp_response(x, 8, 28, 36)]
#[lambda x: 0.0254*modelling_fctns.Wang_Engel_Temp_response(x, 0, 27.5, 40), lambda x: 0.0289*modelling_fctns.Wang_Engel_Temp_response(x, 0, 33, 44)]#0.0254, 0.0289
thresholds = [1, 2]
Maize_set.dev_under_response(lambda x, y: modelling_fctns.phase_dependent_response(x, y, resps, thresholds), 
                             'tas', thresholds[-1])
Maize_set.get_phase_dates(thresholds)
Maize_set.phase_list = ['beginning of flowering', 'yellow ripeness']
Maize_set.get_observed_dataset()#['beginning of emergence', 'beginning of flowering', 'beginning of mil ripeness'])#
Maize_set.get_modelled_dataset()#['beginning of emergence', 'beginning of flowering', 'beginning of mil ripeness'])#
Maize_set.put_obs_data_in_input_array()
Maize_set.compare_modelled_observed()
Maize_set.get_X_y_for_ML('tas', cumulative=True, thinning_parameter=tp)
Maize_set.subsample_X_y(subsample_frac=0.8)
Maize_set.decision_tree(md=3)#['beginning of emergence', 'beginning of flowering', 'beginning of mil ripeness'])#
Maize_set.ML_modelled_observed()

c:\Users\wlwc1989\Documents\Phenology_Test_Notebooks\phenology_dwd\modelling_fctns.py:16: RuntimeWarning: invalid value encountered in power
  f_T = ( ( (2*(T - T_min)**alpha)*((T_opt - T_min)**alpha) - ((T - T_min)**(2*alpha)) ) / ((T_opt - T_min)**(2*alpha)) )**beta


In [16]:
# data rows of csv file
coords = Maize_set.phen_data[['lat', 'lon', 'Stations_id']].drop_duplicates().values
# using the savetxt
# from the numpy module
np.savetxt("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\station_coords.csv",
        coords,
        delimiter =", ",
        fmt ='% s')

In [4]:
def RMSE_ML_model(ds, phases, training_means):
    def RMSE(residuals):
        return np.sqrt(np.mean(residuals**2))
    for phase_index, phase in enumerate(phases):
        print('Phase: ' + str(phase))
        ML_residuals = ds[f'ML prediction emergence to {phase}'] - ds[f'observed time emergence to {phase}']
        ML_residuals = ML_residuals.where(np.abs(ML_residuals) < 60)
        model_residuals = ds[f'modelled time emergence to {phase}'] - ds[f'observed time emergence to {phase}']
        model_residuals = model_residuals.where(np.abs(model_residuals) < 60)
        residuals_to_average = training_means[phase_index] - ds[f'observed time emergence to {phase}']
        residuals_to_average = residuals_to_average.where(np.abs(model_residuals) < 60)
        print('RMSE for ML is ' + str(np.round(RMSE(ML_residuals), decimals=1)))
        print('RMSE for model is ' + str(np.round(RMSE(model_residuals), decimals=1)))
        print('RMSE to training mean is ' + str(np.round(RMSE(residuals_to_average), decimals=1)))

In [12]:
phase_list = ['beginning of flowering', 'yellow ripeness']
training_means = []
for phase in phase_list:
    training_means.append(Maize_set.ds_ML_modelled_observed.where(Maize_set.ds_ML_modelled_observed['Training'] ==True)[f'observed time emergence to {phase}'].mean())
    print(Maize_set.ds_ML_modelled_observed.where(Maize_set.ds_ML_modelled_observed['Training'] ==True)[f'observed time emergence to {phase}'].count())
RMSE_ML_model(Maize_set.ds_ML_modelled_observed.where(Maize_set.ds_ML_modelled_observed['Training'] ==False), phase_list, training_means)
#plot_error_distn(Maize_set.ds_ML_modelled_observed.where(Maize_set.ds_ML_modelled_observed['Training'] ==False), phase_list, training_means)

9872
9872
Phase: beginning of flowering
RMSE for ML is 10.2
RMSE for model is 14.2
RMSE to training mean is 10.2
Phase: yellow ripeness
RMSE for ML is 15.3
RMSE for model is 27.5
RMSE to training mean is 13.7


In [ ]:
phase_list = ['beginning of emergence', 'beginning of flowering', 'beginning of mil ripeness']#['beginning of emergence', 'beginning of flowering', 'beginning of mil ripeness', 'harvest']
#Maize_set.get_observed_dataset(phase_list=phase_list)
plotting.plot_obs_per_year(Maize_set.ds_observed, 'obs_per_year', phase_list=phase_list)
plotting.hist2d_locations(Maize_set.ds_observed.loc[2020].dropna()['lat'], Maize_set.ds_observed.loc[2020].dropna()['lon'], bin_num=5)
plotting.box_plot_modelled_observed(Maize_set.ds_ML_modelled_observed.where(Maize_set.ds_ML_modelled_observed['Training'] ==False), ['beginning of flowering', 'yellow ripeness'])

In [ ]:
tree_rules = tree.export_text(Maize_set.regr, feature_names=[f'Day {tp*i} cum. temp' for i in range(np.int64(np.floor(200/tp)))])
print(Maize_set.y_for_ML)
print(tree_rules)

In [2]:
class Phenology_set:

    phase_names = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phase.txt", encoding = "latin1", engine='python', sep = r';\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;')
    station_data = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phaenologie_Stationen_Jahresmelder.txt",sep = ";\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;", encoding='cp1252', on_bad_lines='skip')

    def __init__(self, address):
        self.phen_data = pd.read_csv(address, encoding = "latin1", engine='python', sep = r';\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;')
        ## CONVERT DATE TO DATETIME ##
        self.phen_data['Eintrittsdatum'] = pd.to_datetime(self.phen_data['Eintrittsdatum'], format = '%Y%m%d')
        self.phen_data = self.phen_data.drop(self.phen_data[self.phen_data['Qualitaetsniveau'] != 10].index)
        self.phase_list = [] #list of phases to consider
        self.add_locations()
        #print(self.phen_data['Qualitaetsniveau'].values)
        self.T_mean = ''
        self.GDD_driver_data = ''
        self.ordered = False

    ### Functions for sorting out dataset ###
    def drop_columns(self, drop_list):
        for drop_name in drop_list:
            self.phen_data = self.phen_data.drop(drop_name, axis = 1)
    
    def phase_order_name(self, stage_order): #[10, 12, 67, 65, 5, 6, 19, 20, 21, 24, ]
        self.phen_data['Order of phase'] = np.nan
        self.phen_data['Name of phase'] = ''
        for i, phaseid in enumerate(stage_order):
            if len(self.phase_names['Phase_englisch'][self.phase_names['Phase_ID'] == str(phaseid)]) != 0:
                #print(i, phaseid)
                self.phen_data.loc[self.phen_data['Phase_id'] == phaseid, 'Order of phase'] = i
                self.phen_data.loc[self.phen_data['Phase_id'] == phaseid, 'Name of phase'] = dataset_fctns.get_phase_name(phaseid, self.phase_names)
        self.order_phen_dataset()

    def order_phen_dataset(self):
        ## SORT BY TIME ##
        if not(np.isin('Order of phase', self.phen_data.columns)):
            print('Get phase order and names first')
        else:
            self.phen_data.sort_values(by = ['Stations_id', 'Referenzjahr', 'Eintrittsdatum', 'Order of phase'])
            self.ordered = True
    
    def get_time_to_next_stage(self):
        #Note phen_data must be time and station ordered. Only plots time to next stage - naive as doesn't consider missing phases.
        if self.ordered:
            ## CALCULATE TIME TO NEXT STAGE ##
            self.phen_data['Time to next stage'] = self.phen_data['Eintrittsdatum'].shift(-1) - self.phen_data['Eintrittsdatum']
            self.phen_data['Next stage name'] = self.phen_data['Name of phase'].shift(-1)
            ## EXCLUDE CHANGES IN STATION ##
            self.phen_data.loc[self.phen_data['Stations_id'] != self.phen_data['Stations_id'].shift(-1), 'Time to next stage'] = np.nan
            self.phen_data.loc[self.phen_data['Stations_id'] != self.phen_data['Stations_id'].shift(-1), 'Next stage name'] = np.nan
        else:
            print('Order dataset so I can get time to next stage')

    def add_locations(self):
        self.phen_data = dataset_fctns.get_station_locations(self.phen_data, self.station_data)
        #LAT, LON = dataset_fctns.get_station_locations(self.phen_data, self.station_data)
        #self.phen_data['lat'] = LAT
        #self.phen_data['lon'] = LON
        #self.phen_data['lat'] = self.phen_data['lat'].map(lambda x: x[0] if isinstance(x, np.float64) == False else x)
        #self.phen_data['lon'] = self.phen_data['lon'].map(lambda x: x[0] if isinstance(x, np.float64) == False else x)
    ### Functions for applying GDD model ###
    def get_mean_T(self, T_address):
        self.T_mean = xr.open_dataset(T_address)

    def index_time_from_emergence_day(self):
        i_day = self.GDD_driver_data['emergence_dates'].values.copy()
        i_daysofyear = np.array([i_day + np.timedelta64(12, 'h') + np.timedelta64(day_of_year, 'D') for day_of_year in range(366)])
        time_indexer = xr.DataArray(i_daysofyear, dims=[ "time", 'modelpoint'])
        self.GDD_driver_data = self.GDD_driver_data.sel(time=time_indexer, method='nearest')

    def align_emergence_obs_with_driver_data(self):
        ## Make sure we are comparing to observations where we have the driver data;
        #1. Align the times - need to check as it might run for some days then go off the end.
        self.just_emergence = self.just_emergence.where(self.just_emergence['Referenzjahr'] <= 2024)
        ## Make sure all elements are in the driver data
        self.just_emergence = self.just_emergence.loc[np.isin(self.just_emergence['Eintrittsdatum'] + np.timedelta64(12, 'h'), self.GDD_driver_data['time'])]
        self.just_emergence = self.just_emergence.loc[np.isin(self.just_emergence['Stations_id'], self.GDD_driver_data['Stations_id'])]
        self.just_emergence = self.just_emergence.dropna()
    
    def get_unique_xy_station(self, x_coords, y_coords, station_ids):
        unique_values = np.unique(np.stack([x_coords, y_coords, station_ids]), axis = 1)
        return unique_values[0, :], unique_values[1, :], unique_values[2, :]

    def make_input_array(self):
        ## Puts pandas phenological frame into driver xarray and aligns the two
        self.just_emergence = self.phen_data.where(self.phen_data['Name of phase'] == 'beginning of emergence').dropna()
        ## For now just do data after 2005 to save time
        self.just_emergence = self.just_emergence.where(self.just_emergence['Eintrittsdatum'] > np.datetime64('2022-01-01')).dropna()
        x_coords = self.just_emergence['lon'].values
        y_coords = self.just_emergence['lat'].values
        station_ids = np.int64(self.just_emergence['Stations_id'].values)
        x_unique, y_unique, stations = self.get_unique_xy_station(x_coords, y_coords, station_ids)
        #Makes an array to put into GDD model
        print('project to new coords')
        x_epsg, y_epsg = dataset_fctns.latlon_to_projection(x_unique, y_unique)
        print('interpolate driver to station locations')
        # Working in xarray (not pandas) after this point:
        self.GDD_driver_data = dataset_fctns.interpolate_xy(x_epsg, y_epsg, self.T_mean)
        self.GDD_driver_data = self.GDD_driver_data.assign_coords(Stations_id=("modelpoint", stations))
        self.GDD_driver_data = self.GDD_driver_data.drop_dims('bnds')
        self.GDD_driver_data = self.GDD_driver_data.set_xindex(['Stations_id'])
        
    
    def dev_under_response(self, response, driver_variable, maturity_t_dev):
        # Response is the rate response to driver values. Driver values are the input to this response. Maturity_t_dev is the t_dev value where we should stop running.
        #self.just_emergence = self.just_emergence.where(self.just_emergence['Referenzjahr'] <= 2019)
        ## Make the indexer to extract things at the right time.
        #self.align_emergence_obs_with_driver_data()
        time_station = xr.Dataset.from_dataframe(self.just_emergence[['Stations_id', 'Eintrittsdatum']])
        time_station = time_station.rename({'index':'Emergence observation', 'Eintrittsdatum':'time'})
        time_station['time'] += np.timedelta64(12, 'h')
        print(time_station)
        ## Initiate development time storage object.
        t_dev = np.zeros(time_station.dims['Emergence observation']) #Continuous development time. When this passes through some thresholds then have change in phase.
        dev_time_series = [t_dev.copy()]
        ## Make sure driver dataset uses station id to index this dimension
        self.GDD_driver_data = self.GDD_driver_data.set_xindex(['Stations_id'])
        #Run model
        for day in range(200):
            print(day)
            driver_values = self.GDD_driver_data.sel(time_station)[driver_variable].values 
            t_dev += response(driver_values, t_dev)
            dev_time_series.append(t_dev.copy())
            time_station['time'] += np.timedelta64(1, 'D')
        dev_time_series.append(self.just_emergence['Eintrittsdatum'].values.astype('datetime64[Y]'))
        dev_time_series.append(self.just_emergence['Stations_id'].values)
        self.model_dev_time_series = np.array(dev_time_series)
        self.GDD_driver_data['Development Time'] = (('days from emergence', 'Emergence observation'), self.model_dev_time_series)

    def get_phase_dates(self, thresholds):
        column_names = np.concatenate([np.array(thresholds), ['Referenzjahr'], ['Stations_id']])
        self.phase_dates_array = np.zeros((len(thresholds), self.model_dev_time_series.shape[1]))
        for obs_index in range(self.model_dev_time_series.shape[1]):
            self.phase_dates_array[:, obs_index] = np.digitize(thresholds, self.model_dev_time_series[:-2, obs_index].astype(np.float64))
        self.phase_dates_array = np.concatenate([self.phase_dates_array, [pd.to_datetime(self.model_dev_time_series[-2]).year], [self.model_dev_time_series[-1]]], axis=0)
        self.phase_dates_array = pd.DataFrame(self.phase_dates_array.T, columns = column_names)
        self.phase_dates_array.set_index(['Referenzjahr', 'Stations_id'])
        #Note that the thresholds are NOT the bins for numpy digitize!
    
    ## Functions for evaluation ##
    def get_observed_dataset(self, winter_sowing = False):
        observed_to_first_stage = dataset_fctns.time_stage_to_stage(self.phen_data, 'beginning of emergence', self.phase_list[0], winter_sowing=winter_sowing).dropna()
        self.ds_observed = pd.DataFrame({f'observed time emergence to {self.phase_list[0]}': observed_to_first_stage})
        for phase in self.phase_list[1:]:
            self.ds_observed[f'observed time emergence to {phase}'] = dataset_fctns.time_stage_to_stage(self.phen_data, 'beginning of emergence', phase, winter_sowing=winter_sowing).dropna()
        self.ds_observed = self.ds_observed.reset_index()
        self.ds_observed = dataset_fctns.get_station_locations(self.ds_observed, self.station_data)
        self.ds_observed = self.ds_observed.merge(self.just_emergence[['Eintrittsdatum', 'Referenzjahr', 'Stations_id']], how = 'outer', on=['Referenzjahr', 'Stations_id']).rename(columns={'Eintrittsdatum':'emergence date'})
        #self.ds_observed = self.ds_observed.set_index(['Referenzjahr', 'Stations_id'])
        #self.ds_observed = pd.concat([self.just_emergence.set_index(['Referenzjahr', 'Stations_id'], inplace=False)['Eintrittsdatum'], self.ds_observed], axis=1).rename(columns={'Eintrittsdatum':'emergence date'})
        #LAT, LON = dataset_fctns.get_station_locations(self.ds_observed, self.station_data)
        #self.ds_observed['lat'] = LAT
        #self.ds_observed['lon'] = LON
        #self.ds_observed['lat'] = self.ds_observed['lat'].map(lambda x: x[0] if isinstance(x, np.float64) == False else x)
        #self.ds_observed['lon'] = self.ds_observed['lon'].map(lambda x: x[0] if isinstance(x, np.float64) == False else x)
    
    def compare_modelled_observed(self):
        self.ds_modelled_observed = pd.concat([self.phase_dates_array, self.ds_observed.drop_duplicates()], axis = 1)

    
    
    def subsample_X_y(self, subsample_frac = 0.5):
        self.subsample = np.random.choice(np.arange(self.y_for_ML.shape[0]),np.int64(np.floor(self.y_for_ML.shape[0]*subsample_frac)))
        self.training_X = self.X_for_ML[self.subsample, :]
        self.training_y = self.y_for_ML[self.subsample, :]
        self.complement_of_subsample = np.delete(np.arange(self.y_for_ML.shape[0]), self.subsample)
        self.verification_X = self.X_for_ML[self.complement_of_subsample, :]
        self.verification_y = self.y_for_ML[self.complement_of_subsample, :]

        self.training_referenzjahr = self.GDD_driver_data['Referenzjahr'].values[self.subsample]
        self.training_stationid = self.GDD_driver_data['Stations_id'].values[self.subsample]
        self.verification_referenzjahr = self.GDD_driver_data['Referenzjahr'].values[self.complement_of_subsample]
        self.verification_stationid = self.GDD_driver_data['Stations_id'].values[self.complement_of_subsample]
    
    def decision_tree(self, md=20):
        self.regr = tree.DecisionTreeRegressor(max_depth=md, min_samples_leaf=5)
        self.fit = self.regr.fit(self.training_X, self.training_y)
        data_ML_training = {'Stations_id': np.int64(self.GDD_driver_data['Stations_id'].values[self.subsample]),
                        'Referenzjahr': np.int64(self.GDD_driver_data['Referenzjahr'].values[self.subsample]),
                        'Training': np.array([True for count in range(len(self.subsample))])
                        }
        data_ML_verification = {'Stations_id': np.int64(self.GDD_driver_data['Stations_id'].values[self.complement_of_subsample]),
                        'Referenzjahr': np.int64(self.GDD_driver_data['Referenzjahr'].values[self.complement_of_subsample]),
                        'Training': np.array([False for count in range(len(self.complement_of_subsample))])
                        }
        self.ds_ML_predictions_training = pd.DataFrame(data_ML_training)
        self.ds_ML_predictions_verification = pd.DataFrame(data_ML_verification)
        #Add modelled phase dates etc. to the comparison set.
        for phase_index, phase in enumerate(self.phase_list):
            self.ds_ML_predictions_training[f'ML prediction emergence to {phase}'] = self.fit.predict(self.training_X)[:, phase_index]
            self.ds_ML_predictions_verification[f'ML prediction emergence to {phase}'] = self.fit.predict(self.verification_X)[:, phase_index]
            self.ds_ML_predictions_training[f'ML check obs to {phase}'] = self.training_y[:, phase_index]
            self.ds_ML_predictions_verification[f'ML check obs to {phase}'] = self.verification_y[:, phase_index]
        self.ds_ML_predictions_training = self.ds_ML_predictions_training.drop_duplicates()
        self.ds_ML_predictions_verification = self.ds_ML_predictions_verification.drop_duplicates()
        self.ds_ML_results = pd.concat([self.ds_ML_predictions_verification, self.ds_ML_predictions_training], axis=0)
        self.ds_ML_results.set_index(['Referenzjahr', 'Stations_id'], inplace=True)
    
    def ML_modelled_observed(self):
        self.ds_ML_modelled_observed = pd.concat([self.ds_ML_results, self.ds_comparison, self.ds_observed], axis = 1)

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_7620\3188619908.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  station_data = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phaenologie_Stationen_Jahresmelder.txt",sep = ";\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;", encoding='cp1252', on_bad_lines='skip')


In [3]:
## Functions for ML ##    
def put_obs_data_in_input_array(self): #, predictor_days = 200
    y_data = {}
    for phase_time in self.ds_observed.columns:
        if phase_time != 'lat' and phase_time != 'lon':
            y_data[phase_time] = (['modelpoint'], self.ds_observed[phase_time].values)
    #print(y_data)
    obs_data = xr.Dataset(data_vars = y_data,
                            coords = {'Referenzjahr':(['modelpoint'], np.int64(self.ds_observed.index.get_level_values(0))),
                                    'Stations_id':(['modelpoint'], np.int64(self.ds_observed.index.get_level_values(1)))}
                                    )
    #print(obs_data)
    obs_data = obs_data.set_xindex(['Referenzjahr', 'Stations_id'])
    self.GDD_driver_data = self.GDD_driver_data.reset_index('modelpoint')
    #self.GDD_driver_data['Stations_id'] = self.GDD_driver_data['Stations_id'].astype(np.int64)
    #self.GDD_driver_data['Referenzjahr'] = self.GDD_driver_data['Referenzjahr'].astype(np.int64)
    self.GDD_driver_data = self.GDD_driver_data.set_coords(['Referenzjahr', 'Stations_id']) #self.data_for_ML = self.data_for_ML.set_coords(['Referenzjahr', 'Stations_id'])
    self.GDD_driver_data = self.GDD_driver_data.set_xindex(['Referenzjahr', 'Stations_id']) #self.data_for_ML = self.data_for_ML.set_xindex(['Referenzjahr', 'Stations_id'])
    self.GDD_driver_data = xr.merge([self.GDD_driver_data, obs_data], join='left') #self.data_for_ML = xr.merge([self.data_for_ML, obs_data], join='left')#[driver_variable]
    self.GDD_driver_data = self.GDD_driver_data.dropna('modelpoint') #self.data_for_ML = self.data_for_ML.dropna('modelpoint')


SyntaxError: incomplete input (3415185510.py, line 23)

In [84]:
Maize_set.GDD_driver_data

<xarray.Dataset> Size: 14MB
Dimensions:             (time: 2963, modelpoint: 600)
Coordinates:
  * time                (time) datetime64[ns] 24kB 2017-01-01T12:00:00 ... 20...
    lon                 (modelpoint) float64 5kB nan nan nan ... 10.06 10.24
    lat                 (modelpoint) float64 5kB nan nan nan ... 47.43 47.71
    x                   (modelpoint) float64 5kB 3.729e+06 ... 4.339e+06
    y                   (modelpoint) float64 5kB 2.678e+06 ... 2.733e+06
  * Stations_id         (modelpoint) float64 5kB 1.948e+04 ... 1.288e+04
Dimensions without coordinates: modelpoint
Data variables:
    crs                 int32 4B 1
    tas                 (time, modelpoint) float64 14MB nan nan nan ... nan 2.7
    number_of_stations  (time) float64 24kB 1.415e+03 1.416e+03 ... 490.0 490.0
Attributes: (12/23)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2017_v6-0_de.nc
    history:                Tue Feb 11 16:27:50 2025: ncrcat tas_hyras_1_2017...
    NCO:                    netCDF Operators version 5.2.7 (Homepage = http:/...

In [71]:
def get_X_y_for_ML(self, driver_variable = 'tas', predictor_days = 200, cumulative = False, thinning_parameter = 1, start_year = 2020, end_year = 2023):
    self.just_emergence = dataset_fctns.add_EOS_to_df(self.just_emergence)
    self.just_emergence = dataset_fctns.add_SOS_to_df(self.just_emergence)
    self.just_emergence['WC SOS date'] = pd.to_datetime(self.just_emergence['Referenzjahr'], format='%Y') + pd.to_timedelta(self.just_emergence['SOS'], 'D')
    self.just_emergence['SOS'] = pd.to_timedelta(self.just_emergence['SOS'], 'D')
    time_station = xr.Dataset.from_dataframe(self.just_emergence[['Stations_id', 'SOS']].drop_duplicates()) #, 'Referenzjahr'
    time_station = time_station.set_coords('Stations_id').set_xindex(['Stations_id'])
    time_station = time_station.drop_vars('index')
    time_station = time_station.expand_dims(dim={'time':pd.to_timedelta(np.arange(0, predictor_days), 'D')})
    time_station = time_station.expand_dims(dim={'Referenzjahr':pd.date_range(f'{start_year}-01-01', periods = end_year - start_year, freq='YS')})
    time_station['SOS'] = time_station['SOS'] + time_station['Referenzjahr'] + time_station['time'] + pd.Timedelta(12, 'h')
    time_station = time_station.rename({'time':'time_from_SOS', 'SOS':'time'})
    time_station = time_station.reset_index('Stations_id').reset_coords(names = 'Stations_id')
    self.driver_data_for_ML = self.GDD_driver_data[driver_variable].sel(time_station)
    self.driver_data_for_ML = self.driver_data_for_ML.rename({'index': 'Stations_id'})
    self.driver_frame_for_ML = self.driver_data_for_ML.to_dataframe(dim_order = ['Referenzjahr', 'Stations_id', 'time_from_SOS'])
    self.driver_frame_for_ML = pd.concat([self.driver_frame_for_ML['tas'].unstack(),
                                          self.driver_frame_for_ML['lat'].unstack()['0 days'].rename('lat'),
                                          self.driver_frame_for_ML['lon'].unstack()['0 days'].rename('lon'),
                                          self.driver_frame_for_ML['time'].unstack()['0 days'].rename('WC SOS')], axis=1)
    self.driver_frame_for_ML.rename(columns={tester.columns[x]: f'{driver_variable} day {x}' for x in range(200)}, inplace=True)
    return self
    #self.X_for_ML = self.driver_data_for_ML[:predictor_days, :].values.T
    #if cumulative:
    #    self.X_for_ML = np.cumsum(self.X_for_ML, axis=1)
    #if thinning_parameter != 1:
    #    self.X_for_ML = self.X_for_ML[:, ::thinning_parameter]
    #self.y_for_ML = np.array([self.GDD_driver_data[phase_time].values for phase_time in self.ds_observed.columns[:-2]])[:predictor_days, :].T
    #for station in self.just_emergence['Stations_id'].unique():
    #    just_station = self.just_emergence.where(self.just_emergence['Stations_id'] == station).dropna()
    #    for year in just_station['Referenzjahr'].unique():
    #        just_station_year = just_station.where(just_station['Referenzjahr'] == year).dropna()
    #        print(just_station_year)
    #        station_data = self.GDD_driver_data.sel({'Stations_id':station})
    #        #, time = slice(just_station_year['WC SOS date'].values[0] + pd.Timedelta(12, 'h'), just_station_year['WC SOS date'].values[0] + pd.Timedelta(12, 'h') + pd.Timedelta(predictor_days, 'D')))
    #        print(station_data)
    return self

In [72]:
hmm = get_X_y_for_ML(Maize_set)

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_7620\1852814812.py:19: UserWarning: rename 'index' to 'Stations_id' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  self.driver_data_for_ML = self.driver_data_for_ML.rename({'index': 'Stations_id'})


In [74]:
hmm.driver_frame_for_ML.dropna()

tas day 0  tas day 1  tas day 2  tas day 3  \
Referenzjahr Stations_id                                               
2020-01-01   6             7.500000  12.468730  13.430438  12.400318   
             385          14.694181  15.119670  12.362744  11.551902   
             386          15.987428  15.600000  11.795632  11.137194   
             387          16.300000  16.585396  15.085396  13.756186   
             388          14.454431  13.959843  12.607356  10.221605   
...                             ...        ...        ...        ...   
2022-01-01   581          10.047645  10.500000  11.584808  11.000000   
             585           9.577661  10.896277   9.545091  10.256730   
             590           8.650209  10.111615   9.037657   9.338594   
             593          11.655101   8.191529   7.973511   8.833178   
             594          12.305412  10.000000   9.722622  10.474207   

                          tas day 4  tas day 5  tas day 6  tas day 7  \
Referenzjahr Stations_id                                               
2020-01-01   6             9.000000   9.483216  10.834206  11.637974   
             385          13.119670  14.032232  12.624118  13.347091   
             386          12.545732  13.691463  12.850328  13.708537   
             387          15.057831  15.530266  14.859475  15.100000   
             388          11.232404  11.949456  11.519661  12.620094   
...                             ...        ...        ...        ...   
2022-01-01   581          11.447645  13.100000  12.500000  10.584808   
             585          12.045091  11.472532   9.110673   9.875346   
             590          11.102009  10.614561   8.589456  10.089456   
             593          10.531976   8.696826   9.499342  10.332520   
             594          11.273108  10.839787  11.482790  11.879619   

                          tas day 8  tas day 9  ...  tas day 193  tas day 194  \
Referenzjahr Stations_id                        ...                             
2020-01-01   6            13.000318  12.055077  ...     9.037974     6.837655   
             385          10.509835  10.952910  ...     7.389157     9.979323   
             386          10.100000  11.025050  ...     6.821109     9.249900   
             387          13.385396  13.085396  ...     9.498355    11.987040   
             388          11.141693  11.141693  ...     5.418155     8.462860   
...                             ...        ...  ...          ...          ...   
2022-01-01   581           9.900000   9.471822  ...    13.437163    16.374326   
             585           9.000910  10.756730  ...    15.590237    15.080475   
             590           9.137657   9.637657  ...    14.838310    15.487448   
             593           9.999342   9.610014  ...    11.522582    13.600000   
             594          12.305412  11.722622  ...    12.948415    14.048415   

                          tas day 195  tas day 196  tas day 197  tas day 198  \
Referenzjahr Stations_id                                                       
2020-01-01   6               6.724639     8.068411     7.707536     9.651627   
             385             9.389157    11.619670    10.889157    12.032232   
             386             8.687522    11.087950    10.821109    11.974950   
             387            11.075779    13.056186    12.604628    13.556186   
             388             8.426992     9.706923     8.452060    10.190712   
...                               ...          ...          ...          ...   
2022-01-01   581            15.875823    14.428178    12.528178    13.269615   
             585            14.460949    11.990237    12.312550     9.500910   
             590            14.600000    11.500000    12.676620     9.300000   
             593            13.738399    17.100658    16.069529    15.467481   
             594            14.000000    16.681818    14.074207    12.900000   

                          tas day 199        lat       lo

In [19]:
test_array = hmm.driver_data_for_ML.dropna(dim='Stations_id').to_dataframe(dim_order = ['Referenzjahr', 'Stations_id', 'time_from_SOS'])#.reset_index()

In [20]:
test_array.index.get_level_values(0)#.to_series()#.str.join('_')

DatetimeIndex(['2020-01-01', '2020-01-01', '2020-01-01', '2020-01-01',
               '2020-01-01', '2020-01-01', '2020-01-01', '2020-01-01',
               '2020-01-01', '2020-01-01',
               ...
               '2022-01-01', '2022-01-01', '2022-01-01', '2022-01-01',
               '2022-01-01', '2022-01-01', '2022-01-01', '2022-01-01',
               '2022-01-01', '2022-01-01'],
              dtype='datetime64[ns]', name='Referenzjahr', length=39600, freq=None)

In [63]:
tester = pd.concat([test_array['tas'].unstack(), 
                    test_array['lat'].unstack()['0 days'].rename('lat'),
                    test_array['lon'].unstack()['0 days'].rename('lon'),
                    test_array['time'].unstack()['0 days'].rename('WC SOS')], axis=1)

In [60]:
type(tester)

pandas.core.frame.DataFrame

In [64]:
tester.rename(columns={tester.columns[x]: f'tas day {x}' for x in range(200)})

tas day 0  tas day 1  tas day 2  tas day 3  \
Referenzjahr Stations_id                                               
2020-01-01   0             7.500000  12.468730  13.430438  12.400318   
             1            14.694181  15.119670  12.362744  11.551902   
             2            15.987428  15.600000  11.795632  11.137194   
             3            16.300000  16.585396  15.085396  13.756186   
             4            14.454431  13.959843  12.607356  10.221605   
...                             ...        ...        ...        ...   
2022-01-01   61           10.047645  10.500000  11.584808  11.000000   
             62            9.577661  10.896277   9.545091  10.256730   
             63            8.650209  10.111615   9.037657   9.338594   
             64           11.655101   8.191529   7.973511   8.833178   
             65           12.305412  10.000000   9.722622  10.474207   

                          tas day 4  tas day 5  tas day 6  tas day 7  \
Referenzjahr Stations_id                                               
2020-01-01   0             9.000000   9.483216  10.834206  11.637974   
             1            13.119670  14.032232  12.624118  13.347091   
             2            12.545732  13.691463  12.850328  13.708537   
             3            15.057831  15.530266  14.859475  15.100000   
             4            11.232404  11.949456  11.519661  12.620094   
...                             ...        ...        ...        ...   
2022-01-01   61           11.447645  13.100000  12.500000  10.584808   
             62           12.045091  11.472532   9.110673   9.875346   
             63           11.102009  10.614561   8.589456  10.089456   
             64           10.531976   8.696826   9.499342  10.332520   
             65           11.273108  10.839787  11.482790  11.879619   

                          tas day 8  tas day 9  ...  tas day 193  tas day 194  \
Referenzjahr Stations_id                        ...                             
2020-01-01   0            13.000318  12.055077  ...     9.037974     6.837655   
             1            10.509835  10.952910  ...     7.389157     9.979323   
             2            10.100000  11.025050  ...     6.821109     9.249900   
             3            13.385396  13.085396  ...     9.498355    11.987040   
             4            11.141693  11.141693  ...     5.418155     8.462860   
...                             ...        ...  ...          ...          ...   
2022-01-01   61            9.900000   9.471822  ...    13.437163    16.374326   
             62            9.000910  10.756730  ...    15.590237    15.080475   
             63            9.137657   9.637657  ...    14.838310    15.487448   
             64            9.999342   9.610014  ...    11.522582    13.600000   
             65           12.305412  11.722622  ...    12.948415    14.048415   

                          tas day 195  tas day 196  tas day 197  tas day 198  \
Referenzjahr Stations_id                                                       
2020-01-01   0               6.724639     8.068411     7.707536     9.651627   
             1               9.389157    11.619670    10.889157    12.032232   
             2               8.687522    11.087950    10.821109    11.974950   
             3              11.075779    13.056186    12.604628    13.556186   
             4               8.426992     9.706923     8.452060    10.190712   
...                               ...          ...          ...          ...   
2022-01-01   61             15.875823    14.428178    12.528178    13.269615   
             62             14.460949    11.990237    12.312550     9.500910   
             63             14.600000    11.500000    12.676620     9.300000   
             64             13.738399    17.100658    16.069529    15.467481   
             65             14.000000    16.681818    14.074207    12.900000   

                          tas day 199        lat       lo

In [49]:
test_array[['lat', 'lon']].columns#unstack()['time_from_SOS']

Index(['lat', 'lon'], dtype='object')

In [31]:
test_array['tas'].unstack()

time_from_SOS                0 days     1 days     2 days     3 days  \
Referenzjahr Stations_id                                               
2020-01-01   0             7.500000  12.468730  13.430438  12.400318   
             1            14.694181  15.119670  12.362744  11.551902   
             2            15.987428  15.600000  11.795632  11.137194   
             3            16.300000  16.585396  15.085396  13.756186   
             4            14.454431  13.959843  12.607356  10.221605   
...                             ...        ...        ...        ...   
2022-01-01   61           10.047645  10.500000  11.584808  11.000000   
             62            9.577661  10.896277   9.545091  10.256730   
             63            8.650209  10.111615   9.037657   9.338594   
             64           11.655101   8.191529   7.973511   8.833178   
             65           12.305412  10.000000   9.722622  10.474207   

time_from_SOS                4 days     5 days     6 days     7 days  \
Referenzjahr Stations_id                                               
2020-01-01   0             9.000000   9.483216  10.834206  11.637974   
             1            13.119670  14.032232  12.624118  13.347091   
             2            12.545732  13.691463  12.850328  13.708537   
             3            15.057831  15.530266  14.859475  15.100000   
             4            11.232404  11.949456  11.519661  12.620094   
...                             ...        ...        ...        ...   
2022-01-01   61           11.447645  13.100000  12.500000  10.584808   
             62           12.045091  11.472532   9.110673   9.875346   
             63           11.102009  10.614561   8.589456  10.089456   
             64           10.531976   8.696826   9.499342  10.332520   
             65           11.273108  10.839787  11.482790  11.879619   

time_from_SOS                8 days     9 days  ...   190 days   191 days  \
Referenzjahr Stations_id                        ...                         
2020-01-01   0            13.000318  12.055077  ...  12.885514  11.807536   
             1            10.509835  10.952910  ...  11.000000   9.842067   
             2            10.100000  11.025050  ...  10.974950   9.500000   
             3            13.385396  13.085396  ...  13.270791  12.072435   
             4            11.141693  11.141693  ...  10.001511   8.620094   
...                             ...        ...  ...        ...        ...   
2022-01-01   61            9.900000   9.471822  ...  15.884808  13.100000   
             62            9.000910  10.756730  ...  13.112079  13.881385   
             63            9.137657   9.637657  ...  12.285574  14.137657   
             64            9.999342   9.610014  ...  13.094702  13.615894   
             65           12.305412  11.722622  ...  14.324694  13.948415   

time_from_SOS              192 days   193 days   194 days   195 days  \
Referenzjahr Stations_id                                               
2020-01-01   0             9.824639   9.037974   6.837655   6.724639   
             1             8.479323   7.389157   9.979323   9.389157   
             2             7.920682   6.821109   9.249900   8.687522   
             3             9.841581   9.498355  11.987040  11.075779   
             4             6.218155   5.418155   8.462860   8.426992   
...                             ...        ...        ...        ...   
2022-01-01   61           14.547645  13.437163  16.374326  15.875823   
             62           13.946057  15.590237  15.080475  14.460949   
             63           13.773021  14.838310  15.487448  14.600000   
             64           13.574245  11.522582  13.600000  13.738399   
             65           14.898901  12.948415  14.048415  14.000000   

time_from_SOS              196 days   197 days   198 days   199 days  
Referenzjahr Stations_id                                              
2020-01-01   0             8.068411   7.70753

In [247]:
test_array.reset_index(level=2)

time_from_SOS                time       lon  \
Referenzjahr Stations_id                                               
2020-12-31   0                  0 days 2021-04-15 12:00:00  6.443151   
             0                  1 days 2021-04-16 12:00:00  6.443151   
             0                  2 days 2021-04-17 12:00:00  6.443151   
             0                  3 days 2021-04-18 12:00:00  6.443151   
             0                  4 days 2021-04-19 12:00:00  6.443151   
...                                ...                 ...       ...   
2022-12-31   65               195 days 2023-10-26 12:00:00  6.474442   
             65               196 days 2023-10-27 12:00:00  6.474442   
             65               197 days 2023-10-28 12:00:00  6.474442   
             65               198 days 2023-10-29 12:00:00  6.474442   
             65               199 days 2023-10-30 12:00:00  6.474442   

                                lat             x             y        tas  
Referenzjahr Stations_id                                                    
2020-12-31   0            50.423130  4.068332e+06  3.040706e+06   0.637974  
             0            50.423130  4.068332e+06  3.040706e+06   2.034206  
             0            50.423130  4.068332e+06  3.040706e+06   3.951309  
             0            50.423130  4.068332e+06  3.040706e+06   5.890433  
             0            50.423130  4.068332e+06  3.040706e+06   6.234206  
...                             ...           ...           ...        ...  
2022-12-31   65           49.894263  4.067758e+06  2.981833e+06   9.347315  
             65           49.894263  4.067758e+06  2.981833e+06  10.422577  
             65           49.894263  4.067758e+06  2.981833e+06  10.722622  
             65           49.894263  4.067758e+06  2.981833e+06  12.056997  
             65           49.894263  4.067758e+06  2.981833e+06  11.913995  

[39600 rows x 7 columns]

In [206]:
Maize_set.GDD_driver_data

<xarray.Dataset> Size: 14MB
Dimensions:             (time: 2963, modelpoint: 600)
Coordinates:
  * time                (time) datetime64[ns] 24kB 2017-01-01T12:00:00 ... 20...
    lon                 (modelpoint) float64 5kB nan nan nan ... 10.06 10.24
    lat                 (modelpoint) float64 5kB nan nan nan ... 47.43 47.71
    x                   (modelpoint) float64 5kB 3.729e+06 ... 4.339e+06
    y                   (modelpoint) float64 5kB 2.678e+06 ... 2.733e+06
  * Stations_id         (modelpoint) float64 5kB 1.948e+04 ... 1.288e+04
Dimensions without coordinates: modelpoint
Data variables:
    crs                 int32 4B 1
    tas                 (time, modelpoint) float64 14MB nan nan nan ... nan 2.7
    number_of_stations  (time) float64 24kB 1.415e+03 1.416e+03 ... 490.0 490.0
Attributes: (12/23)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2017_v6-0_de.nc
    history:                Tue Feb 11 16:27:50 2025: ncrcat tas_hyras_1_2017...
    NCO:                    netCDF Operators version 5.2.7 (Homepage = http:/...

In [201]:
hmm

<xarray.Dataset> Size: 4MB
Dimensions:        (Referenzjahr: 4, time_from_SOS: 200, index: 600)
Coordinates:
  * Referenzjahr   (Referenzjahr) datetime64[ns] 32B 2020-12-31 ... 2023-12-31
  * time_from_SOS  (time_from_SOS) timedelta64[ns] 2kB 0 days ... 199 days
  * Stations_id    (index) float64 5kB 7.521e+03 7.532e+03 ... 1.991e+04
Dimensions without coordinates: index
Data variables:
    time           (Referenzjahr, time_from_SOS, index) datetime64[ns] 4MB 20...

In [196]:
hmm['SOS'] = hmm['SOS'] + hmm['Referenzjahr'] + hmm['time']

In [198]:
hmm['SOS'].values

array([[['2021-04-14T00:00:00.000000000',
         '2021-04-13T00:00:00.000000000',
         '2021-04-14T00:00:00.000000000', ...,
         '2021-04-17T00:00:00.000000000',
         '2021-04-17T00:00:00.000000000',
         '2021-04-15T00:00:00.000000000'],
        ['2021-04-15T00:00:00.000000000',
         '2021-04-14T00:00:00.000000000',
         '2021-04-15T00:00:00.000000000', ...,
         '2021-04-18T00:00:00.000000000',
         '2021-04-18T00:00:00.000000000',
         '2021-04-16T00:00:00.000000000'],
        ['2021-04-16T00:00:00.000000000',
         '2021-04-15T00:00:00.000000000',
         '2021-04-16T00:00:00.000000000', ...,
         '2021-04-19T00:00:00.000000000',
         '2021-04-19T00:00:00.000000000',
         '2021-04-17T00:00:00.000000000'],
        ...,
        ['2021-10-28T00:00:00.000000000',
         '2021-10-27T00:00:00.000000000',
         '2021-10-28T00:00:00.000000000', ...,
         '2021-10-31T00:00:00.000000000',
         '2021-10-31T00:00:00.000000000'

In [139]:
pd.to_timedelta(hmm['obs_index'], 'D')

TypeError: arg must be a string, timedelta, list, tuple, 1-d array, or Series

In [138]:
pd.to_timedelta(hmm['time'], 'D')#.astype('timedelta[D]')

TimedeltaIndex([          '0 days 00:00:00', '0 days 00:00:00.000000001',
                '0 days 00:00:00.000000002', '0 days 00:00:00.000000003',
                '0 days 00:00:00.000000004', '0 days 00:00:00.000000005',
                '0 days 00:00:00.000000006', '0 days 00:00:00.000000007',
                '0 days 00:00:00.000000008', '0 days 00:00:00.000000009',
                ...
                '0 days 00:00:00.000000190', '0 days 00:00:00.000000191',
                '0 days 00:00:00.000000192', '0 days 00:00:00.000000193',
                '0 days 00:00:00.000000194', '0 days 00:00:00.000000195',
                '0 days 00:00:00.000000196', '0 days 00:00:00.000000197',
                '0 days 00:00:00.000000198', '0 days 00:00:00.000000199'],
               dtype='timedelta64[ns]', length=200, freq=None)

In [128]:
for year in hmm['Referenzjahr']:
    for station in hmm['Stations_id']:
        year_station = hmm.sel(Referenzjahr = year, Stations_id = station)
        year_station['obs_index'] = pd.to_timedelta(year_station['obs_index'], 'D')# pd.to_datetime(year_station['Referenzjahr'], format = '%Y') + pd.to_timedelta(year_station['obs_index'], 'D') + pd.Timedelta(12, 'h')
        

MergeError: unable to determine if these variables should be coordinates or not in the merged result: {'obs_index'}

In [133]:
hmm['obs_index'] = pd.to_timedelta(hmm['obs_index'], 'D')#pd.to_datetime(hmm['Referenzjahr'], format = '%Y') #+  #+ pd.to_timedelta(hmm['time'], 'D') #+ pd.Timedelta(12, 'h')

TypeError: arg must be a string, timedelta, list, tuple, 1-d array, or Series

In [116]:
hmm.assign_coords(Referenzjahr = ('Referenzjahr', np.arange(2020, 2024)))

ValueError: cannot re-index or align objects with conflicting indexes found for the following dimensions: 'Station' (2 conflicting indexes)
Conflicting indexes may occur when
- they relate to different sets of coordinate and/or dimension names
- they don't have the same type
- they may be used to reindex data along common dimensions

In [100]:
hmm.expand_dims(dim={'time':200})

<xarray.Dataset> Size: 490kB
Dimensions:      (Station: 600, time: 200)
Coordinates:
  * Stations_id  (Station) float64 5kB 7.521e+03 7.532e+03 ... 1.991e+04
  * Station      (Station) int64 5kB 448 983 3352 4148 ... 407387 407624 411726
Dimensions without coordinates: time
Data variables:
    obs_index    (time, Station) float32 480kB 104.0 103.0 104.0 ... 107.0 105.0

In [93]:
hmm.set_coords('Stations_id').set_xindex(['Stations_id'])

<xarray.Dataset> Size: 14kB
Dimensions:      (index: 600)
Coordinates:
  * Stations_id  (index) float64 5kB 7.521e+03 7.532e+03 ... 1.97e+04 1.991e+04
  * index        (index) int64 5kB 448 983 3352 4148 ... 407387 407624 411726
Data variables:
    WC SOS date  (index) datetime64[ns] 5kB 2022-04-15 2022-04-14 ... 2022-04-16

In [83]:
hmm.driver_data_for_ML.assign_coords(Referenzjahr = ('Emergence observation', pd.to_datetime(hmm.driver_data_for_ML['time'].isel(time=1)).year))

<xarray.DataArray 'tas' (time: 200, Emergence observation: 600)> Size: 960kB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
    time                   (time, Emergence observation) datetime64[ns] 960kB ...
    lon                    (Emergence observation) float64 5kB 5.303 ... 5.241
    lat                    (Emergence observation) float64 5kB 50.57 ... 46.58
    x                      (Emergence observation) float64 5kB 3.988e+06 ... ...
    y                      (Emergence observation) float64 5kB 3.062e+06 ... ...
    Stations_id            (Emergence observation) float64 5kB 7.521e+03 ... ...
  * Emergence observation  (Emergence observation) int64 5kB 448 983 ... 411726
    Referenzjahr           (Emergence observation) int32 2kB 2022 2022 ... 2022
Attributes:
    standard_name:      air_temperature
    long_name:          Daily Mean Air Temperature
    units:              degree_Celsius
    grid_mapping:       crs
    cell_methods:       time: mean
    units_metadata:     temperature: on-scale
    CoordinateSystems:  LatLonCoordinateSystem ProjectionCoordinateSystem
    esri_pe_string:     PROJCS["ETRS_1989_LAEA",GEOGCS["GCS_ETRS_1989",DATUM[...

In [81]:
pd.to_datetime(hmm.driver_data_for_ML['time'].isel(time=1)).year

Index([2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022,
       ...
       2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022],
      dtype='int32', length=600)

In [79]:
hmm.driver_data_for_ML['time']

<xarray.DataArray 'time' (time: 200, Emergence observation: 600)> Size: 960kB
array([['2022-02-24T12:00:00.000000000', '2022-02-23T12:00:00.000000000',
        '2022-02-24T12:00:00.000000000', ...,
        '2022-02-27T12:00:00.000000000', '2022-02-27T12:00:00.000000000',
        '2022-02-25T12:00:00.000000000'],
       ['2022-02-25T12:00:00.000000000', '2022-02-24T12:00:00.000000000',
        '2022-02-25T12:00:00.000000000', ...,
        '2022-02-28T12:00:00.000000000', '2022-02-28T12:00:00.000000000',
        '2022-02-26T12:00:00.000000000'],
       ['2022-02-26T12:00:00.000000000', '2022-02-25T12:00:00.000000000',
        '2022-02-26T12:00:00.000000000', ...,
        '2022-03-01T12:00:00.000000000', '2022-03-01T12:00:00.000000000',
        '2022-02-27T12:00:00.000000000'],
       ...,
       ['2022-09-09T12:00:00.000000000', '2022-09-08T12:00:00.000000000',
        '2022-09-09T12:00:00.000000000', ...,
        '2022-09-12T12:00:00.000000000', '2022-09-12T12:00:00.000000000',
        '2022-09-10T12:00:00.000000000'],
       ['2022-09-10T12:00:00.000000000', '2022-09-09T12:00:00.000000000',
        '2022-09-10T12:00:00.000000000', ...,
        '2022-09-13T12:00:00.000000000', '2022-09-13T12:00:00.000000000',
        '2022-09-11T12:00:00.000000000'],
       ['2022-09-11T12:00:00.000000000', '2022-09-10T12:00:00.000000000',
        '2022-09-11T12:00:00.000000000', ...,
        '2022-09-14T12:00:00.000000000', '2022-09-14T12:00:00.000000000',
        '2022-09-12T12:00:00.000000000']], dtype='datetime64[ns]')
Coordinates:
    time                   (time, Emergence observation) datetime64[ns] 960kB ...
    lon                    (Emergence observation) float64 5kB 5.303 ... 5.241
    lat                    (Emergence observation) float64 5kB 50.57 ... 46.58
    x                      (Emergence observation) float64 5kB 3.988e+06 ... ...
    y                      (Emergence observation) float64 5kB 3.062e+06 ... ...
    Stations_id            (Emergence observation) float64 5kB 7.521e+03 ... ...
  * Emergence observation  (Emergence observation) int64 5kB 448 983 ... 411726
Attributes:
    standard_name:  time
    long_name:      Mid Of Twentyfour Hour Time Interval [UTC]
    bounds:         time_bnds
    axis:           T

In [48]:
hmm

<xarray.Dataset> Size: 19kB
Dimensions:       (index: 600)
Coordinates:
  * index         (index) int64 5kB 448 983 3352 4148 ... 407387 407624 411726
Data variables:
    Stations_id   (index) float64 5kB 7.521e+03 7.532e+03 ... 1.97e+04 1.991e+04
    WC SOS date   (index) datetime64[ns] 5kB 2022-04-15 ... 2022-04-16
    Referenzjahr  (index) float64 5kB 2.022e+03 2.022e+03 ... 2.022e+03

In [26]:
pd.to_datetime(hmm.just_emergence['Referenzjahr'], format='%Y') + pd.to_timedelta(hmm.just_emergence['SOS'], 'D')

448      2022-04-15
983      2022-04-14
3352     2022-04-15
4148     2022-04-15
4973     2022-04-14
            ...    
406854   2022-04-17
407154   2022-04-17
407387   2022-04-18
407624   2022-04-18
411726   2022-04-16
Length: 600, dtype: datetime64[ns]

In [22]:
years = hmm.just_emergence['Referenzjahr']
hmm.just_emergence['WC season start'] = pd.DatetimeIndex(f'{np.int64(years)}-01-01') + pd.Timedelta(hmm.just_emergence['SOS'], 'D')#[['Stations_id', 'Referenzjahr']]

TypeError: DatetimeIndex(...) must be called with a collection of some kind, '[2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022\n 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 2022 20

In [ ]:
    time_station = xr.Dataset.from_dataframe(self.just_emergence[['Stations_id', 'Eintrittsdatum']])
    time_station = time_station.rename({'index':'Emergence observation', 'Eintrittsdatum':'obs_index'})
    time_station['obs_index'] += (np.timedelta64(-49, 'D') + np.timedelta64(-12, 'h'))
    time_station['obs_index'] = time_station['obs_index'].expand_dims(dim={'time':365}).copy()
    time_station = time_station.rename({'obs_index':'time'})
    #time_station.assign_coords({'time_for_inputs':time_station['time']})
    for n in range(365):
        time_station['time'][n] += np.timedelta64(n, 'D')
    print(time_station)
    self.driver_data_for_ML = self.GDD_driver_data[driver_variable].sel(time_station)
    self.X_for_ML = self.driver_data_for_ML[:predictor_days, :].values.T
    if cumulative:
        self.X_for_ML = np.cumsum(self.X_for_ML, axis=1)
    if thinning_parameter != 1:
        self.X_for_ML = self.X_for_ML[:, ::thinning_parameter]
    self.y_for_ML = np.array([self.GDD_driver_data[phase_time].values for phase_time in self.ds_observed.columns[:-2]])[:predictor_days, :].T

In [22]:
Maize_set.GDD_driver_data

<xarray.Dataset> Size: 26MB
Dimensions:             (time: 2963, modelpoint: 835, days from emergence: 203,
                         Emergence observation: 3880)
Coordinates:
  * time                (time) datetime64[ns] 24kB 2017-01-01T12:00:00 ... 20...
    lon                 (modelpoint) float64 7kB nan nan nan ... 10.06 10.24
    lat                 (modelpoint) float64 7kB nan nan nan ... 47.43 47.71
    x                   (modelpoint) float64 7kB 3.735e+06 ... 4.339e+06
    y                   (modelpoint) float64 7kB 2.784e+06 ... 2.733e+06
  * Stations_id         (modelpoint) float64 7kB 8.776e+03 ... 1.288e+04
Dimensions without coordinates: modelpoint, days from emergence,
                                Emergence observation
Data variables:
    crs                 int32 4B 1
    tas                 (time, modelpoint) float64 20MB nan nan nan ... nan 2.7
    number_of_stations  (time) float64 24kB 1.415e+03 1.416e+03 ... 490.0 490.0
    Development Time    (days from emergence, Emergence observation) object 6MB ...
Attributes: (12/23)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2017_v6-0_de.nc
    history:                Tue Feb 11 16:27:50 2025: ncrcat tas_hyras_1_2017...
    NCO:                    netCDF Operators version 5.2.7 (Homepage = http:/...

In [20]:
hmm = get_observed_dataset(Maize_set)

In [21]:
hmm.ds_observed

,Referenzjahr,Stations_id,observed time emergence to beginning of flowering,observed time emergence to yellow ripeness,lat,lon,emergence date
0,1951.0,7617.0,29.0,NaN,54.6167,9.0333,NaT
1,1951.0,7899.0,80.0,NaN,52.0833,9.5667,NaT
2,1951.0,8137.0,50.0,NaN,52.8667,9.6000,NaT
3,1951.0,8191.0,54.0,NaN,53.2667,10.1167,NaT
4,1951.0,8234.0,98.0,NaN,52.9333,11.1167,NaT
...,...,...,...,...,...,...,...
28330,2022.0,19587.0,60.0,NaN,48.7091,9.2147,2022-05-17
28331,2022.0,19588.0,NaN,NaN,NaN,NaN,2022-05-11
28332,2022.0,19699.0,NaN,NaN,NaN,NaN,2022-05-07
28333,2022.0,19700.0,NaN,NaN,NaN,NaN,2022-05-12


In [52]:
MS = get_X_y_for_ML(Maize_set)#.copy()

<xarray.Dataset> Size: 29MB
Dimensions:                (Emergence observation: 9915, time: 365)
Coordinates:
  * Emergence observation  (Emergence observation) int64 79kB 88 97 ... 411712
Dimensions without coordinates: time
Data variables:
    Stations_id            (Emergence observation) float64 79kB 7.504e+03 ......
    time                   (time, Emergence observation) datetime64[ns] 29MB ...


In [39]:
tt['time'][1] = tt['time'][1] + np.timedelta64(1, 'D')

In [26]:
#Maize_set.just_emergence = Maize_set.just_emergence.where(Maize_set.just_emergence['Referenzjahr'] <= 2020)
time_station = xr.Dataset.from_dataframe(Maize_set.just_emergence[['Stations_id', 'Eintrittsdatum']])
time_station = time_station.rename({'index':'Emergence observation', 'Eintrittsdatum':'time'})
time_station['time'] += np.timedelta64(12, 'h')
time_station = time_station.where(np.isin(time_station['time'], Maize_set.GDD_driver_data['time']))
time_station = time_station.where(np.isin(time_station['Stations_id'], Maize_set.GDD_driver_data['Stations_id']))
time_station = time_station.dropna(dim='Emergence observation')
driver_values = self.GDD_driver_data.sel(time_station)
#Maize_set.GDD_driver_data = Maize_set.GDD_driver_data.rename({'modelpoint':'Stations_id'}).set_index(Stations_id='Stations_id')
#time_station_indexer = xr.DataArray(time_station, dims=['modelpoint'])

In [4]:
Maize_set = Phenology_set("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\PH_Jahresmelder_Landwirtschaft_Kulturpflanze_Mais_1936_2023_hist.txt")
Maize_set.drop_columns(['Unnamed: 9'])
Maize_set.phase_order_name([10, 12, 67, 65, 5, 6, 19, 20, 21, 24, ])
Maize_set.get_mean_T('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\hyras_meantemp_2017_2025.nc') #C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\tas_hyras_5_1951_2020_v5-0_de.nc')
Maize_set.make_input_array()
Maize_set.phase_list = ['beginning of flowering', 'yellow ripeness']
tp=3
resps = [lambda x: 0.053*modelling_fctns.Wang_Engel_Temp_response(x, 8, 28, 36), lambda x: 0.055*modelling_fctns.Wang_Engel_Temp_response(x, 8, 28, 36)]
#[lambda x: 0.0254*modelling_fctns.Wang_Engel_Temp_response(x, 0, 27.5, 40), lambda x: 0.0289*modelling_fctns.Wang_Engel_Temp_response(x, 0, 33, 44)]#0.0254, 0.0289
thresholds = [1, 2]


project to new coords
interpolate driver to station locations


In [5]:
Maize_set.dev_under_response(lambda x, y: modelling_fctns.phase_dependent_response(x, y, resps, thresholds), 
                             'tas', thresholds[-1])
Maize_set.get_phase_dates(thresholds)
Maize_set.get_observed_dataset()
Maize_set.compare_modelled_observed()

<xarray.Dataset> Size: 14kB
Dimensions:                (Emergence observation: 600)
Coordinates:
  * Emergence observation  (Emergence observation) int64 5kB 448 983 ... 411726
Data variables:
    Stations_id            (Emergence observation) float64 5kB 7.521e+03 ... ...
    time                   (Emergence observation) datetime64[ns] 5kB 2022-05...


C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_7620\3188619908.py:113: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  t_dev = np.zeros(time_station.dims['Emergence observation']) #Continuous development time. When this passes through some thresholds then have change in phase.


ValueError: those coordinates already have an index: {'Stations_id'}

In [9]:
Maize_set.ds_modelled_observed

,1,2,Referenzjahr,Stations_id,emergence date,observed time emergence to beginning of flowering,observed time emergence to yellow ripeness,lat,lon
0,201.0,201.0,2017,7521.0,NaT,NaN,NaN,NaN,NaN
1,201.0,201.0,2018,7521.0,NaT,NaN,NaN,NaN,NaN
2,201.0,201.0,2019,7521.0,NaT,NaN,NaN,NaN,NaN
3,201.0,201.0,2020,7521.0,NaT,NaN,NaN,NaN,NaN
4,201.0,201.0,2021,7521.0,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
"(2016.0, 19727.0)",NaN,NaN,NaN,NaN,NaT,76.0,147.0,51.3873,8.5737
"(2016.0, 19729.0)",NaN,NaN,NaN,NaN,NaT,72.0,142.0,52.1014,11.2322
"(2016.0, 19731.0)",NaN,NaN,NaN,NaN,NaT,64.0,105.0,50.8969,13.2205
"(2016.0, 19755.0)",NaN,NaN,NaN,NaN,NaT,102.0,160.0,51.2873,11.0183


In [7]:
df = Maize_set.ds_observed.reset_index()
df.where(df['Referenzjahr'] == 2020).dropna()

,Referenzjahr,Stations_id,emergence date,observed time emergence to beginning of flowering,observed time emergence to yellow ripeness,lat,lon
3,2020.0,7521.0,2020-04-29,91.0,166.0,54.4333,9.8167
9,2020.0,7532.0,2020-05-12,103.0,141.0,54.4000,8.7667
29,2020.0,7616.0,2020-05-10,73.0,134.0,54.6167,8.9667
34,2020.0,7623.0,2020-05-05,78.0,133.0,54.6667,9.1333
45,2020.0,7642.0,2020-05-10,90.0,133.0,54.2333,11.0833
...,...,...,...,...,...,...,...
3820,2020.0,19519.0,2020-05-04,88.0,134.0,54.0027,11.1908
3829,2020.0,19554.0,2020-05-08,95.0,115.0,53.3222,11.9321
3842,2020.0,19585.0,2020-05-10,76.0,132.0,53.6865,11.0608
3848,2020.0,19587.0,2020-05-08,76.0,124.0,48.7091,9.2147


In [3]:
ds = xr.open_dataset('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\tas_hyras_1_2017_v6-0_de.nc')

In [4]:
ds2 = xr.open_dataset('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\tas_hyras_1_2019_v6-0_de.nc')

In [22]:
ds['tas']

AttributeError: 'DataArray' object has no attribute 'iloc'

In [15]:
ds_list = []
for year in range(2017, 2025):
    ds_list.append(xr.open_dataset(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\tas_hyras_1_{year}_v6-0_de.nc'))


In [16]:
len(ds_list)

8

In [24]:
ds = xr.open_dataset('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\hyras_meantemp_2017_2025.nc')

In [29]:
ds.isel({'x':500, 'y':500, 'time': 1000})['tas'].values

array(13.6000002)

In [18]:
ds_full = xr.concat(ds_list, dim='time')

In [23]:
ds_full.drop_dims('bnds')#.to_netcdf('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\hyras_mean_temp.nc')

<xarray.Dataset> Size: 31GB
Dimensions:             (time: 2922, y: 1100, x: 1200)
Coordinates:
  * time                (time) datetime64[ns] 23kB 2017-01-01T12:00:00 ... 20...
    lon                 (y, x) float64 11MB 3.43 3.443 3.455 ... 20.68 20.7
    lat                 (y, x) float64 11MB 45.29 45.29 45.29 ... 54.92 54.92
  * x                   (x) float32 5kB 3.806e+06 3.806e+06 ... 5.004e+06
  * y                   (y) float32 4kB 2.486e+06 2.488e+06 ... 3.586e+06
Data variables:
    crs                 (time) int32 12kB 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    tas                 (time, y, x) float64 31GB nan nan nan ... nan nan nan
    number_of_stations  (time) float64 23kB 1.415e+03 1.416e+03 ... 490.0 490.0
Attributes: (12/21)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    license:                The HYRAS data, produced by DWD, is licensed unde...
    ConventionsURL:         http://cfconventions.org/Data/cf-conventions/cf-c...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2017_v6-0_de.nc

In [19]:
ds_full.to_netcdf('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\hyras_mean_temp.nc')

MemoryError: Unable to allocate 14.4 GiB for an array with shape (2922, 1100, 1200) and data type int32

In [11]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:             (time: 365, bnds: 2, y: 1100, x: 1200)
Coordinates:
  * time                (time) datetime64[ns] 3kB 2017-01-01T12:00:00 ... 201...
    lon                 (y, x) float64 11MB ...
    lat                 (y, x) float64 11MB ...
  * x                   (x) float32 5kB 3.806e+06 3.806e+06 ... 5.004e+06
  * y                   (y) float32 4kB 2.486e+06 2.488e+06 ... 3.586e+06
Dimensions without coordinates: bnds
Data variables:
    time_bnds           (time, bnds) datetime64[ns] 6kB ...
    x_bnds              (x, bnds) float32 10kB ...
    y_bnds              (y, bnds) float32 9kB ...
    crs                 int32 4B ...
    tas                 (time, y, x) float64 4GB ...
    number_of_stations  (time) float64 3kB ...
Attributes: (12/21)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    license:                The HYRAS data, produced by DWD, is licensed unde...
    ConventionsURL:         http://cfconventions.org/Data/cf-conventions/cf-c...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2017_v6-0_de.nc

In [6]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:             (time: 365, bnds: 2, y: 1100, x: 1200)
Coordinates:
  * time                (time) datetime64[ns] 3kB 2017-01-01T12:00:00 ... 201...
    lon                 (y, x) float64 11MB ...
    lat                 (y, x) float64 11MB ...
  * x                   (x) float32 5kB 3.806e+06 3.806e+06 ... 5.004e+06
  * y                   (y) float32 4kB 2.486e+06 2.488e+06 ... 3.586e+06
Dimensions without coordinates: bnds
Data variables:
    time_bnds           (time, bnds) datetime64[ns] 6kB ...
    x_bnds              (x, bnds) float32 10kB ...
    y_bnds              (y, bnds) float32 9kB ...
    crs                 int32 4B ...
    tas                 (time, y, x) float64 4GB ...
    number_of_stations  (time) float64 3kB ...
Attributes: (12/21)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    license:                The HYRAS data, produced by DWD, is licensed unde...
    ConventionsURL:         http://cfconventions.org/Data/cf-conventions/cf-c...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2017_v6-0_de.nc

In [7]:
ds3 = xr.concat([ds, ds2], dim='time')

In [14]:
ds3.isel({'x':500, 'y':500})

<xarray.Dataset> Size: 44kB
Dimensions:             (time: 730, bnds: 2)
Coordinates:
  * time                (time) datetime64[ns] 6kB 2017-01-01T12:00:00 ... 201...
    lon                 float64 8B 9.784
    lat                 float64 8B 49.99
    x                   float32 4B 4.306e+06
    y                   float32 4B 2.986e+06
Dimensions without coordinates: bnds
Data variables:
    time_bnds           (time, bnds) datetime64[ns] 12kB 2017-01-01 ... 2020-...
    x_bnds              (time, bnds) float32 6kB 4.305e+06 ... 4.306e+06
    y_bnds              (time, bnds) float32 6kB 2.986e+06 ... 2.987e+06
    crs                 (time) int32 3kB 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    tas                 (time) float64 6kB -5.2 -1.5 -0.2 0.8 ... -2.8 -2.2 -1.0
    number_of_stations  (time) float64 6kB 1.415e+03 1.416e+03 ... 1.474e+03
Attributes: (12/21)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    license:                The HYRAS data, produced by DWD, is licensed unde...
    ConventionsURL:         http://cfconventions.org/Data/cf-conventions/cf-c...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2017_v6-0_de.nc

In [23]:
xr.concat([ds, ds2], dim='time').dropna()

TypeError: Dataset.dropna() missing 1 required positional argument: 'dim'

In [30]:
ds2

<xarray.Dataset> Size: 4GB
Dimensions:             (time: 365, y: 1100, x: 1200)
Coordinates:
  * time                (time) datetime64[ns] 3kB 2019-01-01T12:00:00 ... 201...
    lon                 (y, x) float64 11MB ...
    lat                 (y, x) float64 11MB ...
  * x                   (x) float32 5kB 3.806e+06 3.806e+06 ... 5.004e+06
  * y                   (y) float32 4kB 2.486e+06 2.488e+06 ... 3.586e+06
Data variables:
    crs                 int32 4B ...
    tas                 (time, y, x) float64 4GB ...
    number_of_stations  (time) float64 3kB ...
Attributes: (12/21)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    license:                The HYRAS data, produced by DWD, is licensed unde...
    ConventionsURL:         http://cfconventions.org/Data/cf-conventions/cf-c...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2019_v6-0_de.nc

In [31]:
ds3 = xr.merge([ds,ds2], join = 'outer')

: 

In [25]:
ds3

<xarray.Dataset> Size: 8GB
Dimensions:             (time: 730, x: 1200, y: 1100, bnds: 2)
Coordinates:
  * time                (time) datetime64[ns] 6kB 2017-01-01T12:00:00 ... 201...
  * x                   (x) float32 5kB 3.806e+06 3.806e+06 ... 5.004e+06
  * y                   (y) float32 4kB 2.486e+06 2.488e+06 ... 3.586e+06
    lon                 (y, x) float64 11MB 3.43 3.443 3.455 ... 20.68 20.7
    lat                 (y, x) float64 11MB 45.29 45.29 45.29 ... 54.92 54.92
Dimensions without coordinates: bnds
Data variables:
    time_bnds           (time, bnds) datetime64[ns] 12kB 2017-01-01 ... 2020-...
    x_bnds              (x, bnds) float32 10kB 3.805e+06 3.806e+06 ... 5.005e+06
    y_bnds              (y, bnds) float32 9kB 2.486e+06 2.487e+06 ... 3.586e+06
    crs                 int32 4B 1
    tas                 (time, y, x) float64 8GB nan nan nan nan ... nan nan nan
    number_of_stations  (time) float64 6kB 1.415e+03 1.416e+03 ... 1.474e+03
Attributes: (12/21)
    source:                 surface observations
    institution:            Deutscher Wetterdienst (DWD)
    Conventions:            CF-1.11
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    realization:            v6-0
    project_id:             HYRAS
    ...                     ...
    license:                The HYRAS data, produced by DWD, is licensed unde...
    ConventionsURL:         http://cfconventions.org/Data/cf-conventions/cf-c...
    realm:                  atmos
    product:                observations
    input_data_status:      checked
    filename:               tas_hyras_1_2017_v6-0_de.nc

In [10]:
df = Maize_set.just_emergence
df.where(df['Referenzjahr'] == 2020).dropna()

,Stations_id,Referenzjahr,Qualitaetsniveau,Objekt_id,Phase_id,Eintrittsdatum,Eintrittsdatum_QB,Jultag,eor,lat,lon,Order of phase,Name of phase
430,7521.0,2020.0,10.0,215.0,12.0,2020-04-29,1.0,119.0,eor,54.4333,9.8167,1.0,beginning of emergence
965,7532.0,2020.0,10.0,215.0,12.0,2020-05-12,1.0,132.0,eor,54.4000,8.7667,1.0,beginning of emergence
3338,7592.0,2020.0,10.0,215.0,12.0,2020-05-03,1.0,123.0,eor,53.4833,10.6333,1.0,beginning of emergence
4137,7608.0,2020.0,10.0,215.0,12.0,2020-05-10,1.0,130.0,eor,53.5000,10.4833,1.0,beginning of emergence
4499,7616.0,2020.0,10.0,215.0,12.0,2020-05-10,1.0,130.0,eor,54.6167,8.9667,1.0,beginning of emergence
...,...,...,...,...,...,...,...,...,...,...,...,...,...
406458,19585.0,2020.0,10.0,215.0,12.0,2020-05-10,1.0,130.0,eor,53.6865,11.0608,1.0,beginning of emergence
406836,19587.0,2020.0,10.0,215.0,12.0,2020-05-08,1.0,128.0,eor,48.7091,9.2147,1.0,beginning of emergence
407381,19699.0,2020.0,10.0,215.0,12.0,2020-04-29,1.0,119.0,eor,51.0418,11.0062,1.0,beginning of emergence
407609,19700.0,2020.0,10.0,215.0,12.0,2020-05-17,1.0,137.0,eor,50.4316,11.9443,1.0,beginning of emergence


In [30]:
je = Maize_set.just_emergence.reset_index()
je.where(je['Stations_id'] == 19488).dropna()

,index,Stations_id,Referenzjahr,Qualitaetsniveau,Objekt_id,Phase_id,Eintrittsdatum,Eintrittsdatum_QB,Jultag,eor,lat,lon,Order of phase,Name of phase
10246,402300.0,19488.0,2007.0,10.0,215.0,12.0,2007-05-08,1.0,128.0,eor,48.4024,11.6946,1.0,beginning of emergence
10247,402309.0,19488.0,2008.0,10.0,215.0,12.0,2008-05-16,1.0,136.0,eor,48.4024,11.6946,1.0,beginning of emergence
10248,402318.0,19488.0,2009.0,10.0,215.0,12.0,2009-05-06,1.0,126.0,eor,48.4024,11.6946,1.0,beginning of emergence
10249,402327.0,19488.0,2010.0,10.0,215.0,12.0,2010-05-04,1.0,124.0,eor,48.4024,11.6946,1.0,beginning of emergence
10250,402336.0,19488.0,2011.0,10.0,215.0,12.0,2011-05-10,1.0,130.0,eor,48.4024,11.6946,1.0,beginning of emergence
10251,402345.0,19488.0,2012.0,10.0,215.0,12.0,2012-05-10,1.0,130.0,eor,48.4024,11.6946,1.0,beginning of emergence
10252,402354.0,19488.0,2013.0,10.0,215.0,12.0,2013-05-11,1.0,131.0,eor,48.4024,11.6946,1.0,beginning of emergence
10253,402362.0,19488.0,2014.0,10.0,215.0,12.0,2014-05-07,1.0,127.0,eor,48.4024,11.6946,1.0,beginning of emergence
10254,402371.0,19488.0,2015.0,10.0,215.0,12.0,2015-05-07,1.0,127.0,eor,48.4024,11.6946,1.0,beginning of emergence
10255,402380.0,19488.0,2016.0,10.0,215.0,12.0,2016-05-10,1.0,130.0,eor,48.4024,11.6946,1.0,beginning of emergence


In [34]:
Maize_set.ds_observed

emergence date  \
Referenzjahr Stations_id                  
2007.0       7504.0          2007-05-02   
2008.0       7504.0          2008-05-07   
2009.0       7504.0          2009-05-15   
2010.0       7504.0          2010-05-19   
2011.0       7504.0          2011-04-30   
...                                 ...   
2006.0       19735.0                NaT   
             19739.0                NaT   
             19740.0                NaT   
             19741.0                NaT   
             19743.0                NaT   

                          observed time emergence to beginning of flowering  \
Referenzjahr Stations_id                                                      
2007.0       7504.0                                                    80.0   
2008.0       7504.0                                                    77.0   
2009.0       7504.0                                                    83.0   
2010.0       7504.0                                                    71.0   
2011.0       7504.0                                                    89.0   
...                                                                     ...   
2006.0       19735.0                                                   80.0   
             19739.0                                                   56.0   
             19740.0                                                   66.0   
             19741.0                                                   84.0   
             19743.0                                                   88.0   

                          observed time emergence to yellow ripeness      lat  \
Referenzjahr Stations_id                                                        
2007.0       7504.0                                            145.0  54.3167   
2008.0       7504.0                                            147.0  54.3167   
2009.0       7504.0                                            126.0  54.3167   
2010.0       7504.0                                            140.0  54.3167   
2011.0       7504.0                                            147.0  54.3167   
...                                                              ...      ...   
2006.0       19735.0                                           125.0  50.9352   
             19739.0                                           122.0  49.7167   
             19740.0                                             NaN  52.8719   
             19741.0                                           107.0  52.1458   
             19743.0                                           125.0  51.6070   

                              lon  
Referenzjahr Stations_id           
2007.0       7504.0       10.0500  
2008.0       7504.0       10.0500  
2009.0       7504.0       10.0500  
2010.0       7504.0       10.0500  
2011.0       7504.0       10.0500  
...                           ...  
2006.0       19735.0      12.6206  
             19739.0       8.1167  
             19740.0      13.3927  
             19741.0      11.9563  
             19743.0      11.1183  

[29150 rows x 5 columns]

In [14]:
# data rows of csv file
coords = Maize_set.just_emergence[['lat', 'lon', 'Stations_id']].drop_duplicates().values
# using the savetxt
# from the numpy module
np.savetxt("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\station_coords_shortest.csv",
        coords,
        delimiter =", ",
        fmt ='% s')

In [13]:
coords.shape

(835, 3)

In [9]:
coords.shape

(1208, 3)

In [35]:
observed_dataset = Maize_set.ds_observed.reset_index()
observed_dataset = observed_dataset.where(observed_dataset['Referenzjahr'] >=2017).drop_duplicates()

In [36]:
observed_dataset.to_csv('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\observed_times.csv')